In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
import os
import itertools
import pandas as pd
import gvar as gv
from importlib import reload

# Patrick imports
from utils import *
import sys
sys.path.append('/Users/theoares/lqcd/utilities')
from fittools import *
from formattools import *
import plottools as pt

style = styles['talk']

pt.set_font()

#### Input parameters

In [2]:
paper = 'hex-dibaryon'
# paper = 'physical'
# paper = 'variational'         # paper to use input from

In [3]:
# M0v  = 0.02155
M0v   = 0.026479                 # our input, depending on the fit (GeV^2).
ZV    = 0.802                    # vector renormalization.
L, T  = 32, 48                   # lattice size
mpi   = 0.806                    # GeV
gA    = gv.gvar('1.13(7)')       # axial coupling

# Unit conversions
hbarc = .197327                  # GeV*fm
def fm_to_GeV(a):
    return a / hbarc
def to_mpi_units(mass_GeV):
    """Converts a mass in GeV (mass_GeV) to mpi = 1 units."""
    return mass_GeV / mpi
def to_GeV(mass_Mpi):
    """Converts a mass in mpi = 1 units (mass_Mpi) to GeV."""
    return mass_Mpi * mpi

In [4]:
# Paper-specific inputs (physical units)
a_lat = {
    'variational'  : gv.gvar('0.1453(16)'),
    'hex-dibaryon' : gv.gvar('0.1453(16)'),
    'physical'     : -1,
}[paper]                     # lattice spacing (fm)
ainv_gev = {
    'variational'  : -1,
    'hex-dibaryon' : gv.gvar('0.44(9)') * hbarc,          # Eq. (19) in hex-dibaryon
    'physical'     : -1,
}[paper]                     # inverse SCATTERING LENGTH (GeV)
ainv_mpi = {
    'variational'  : gv.gvar('-0.0789(480)'),
    'hex-dibaryon' : to_mpi_units(ainv_gev),
    'physical'     : -1,
}[paper]                     # inverse SCATTERING LENGTH (mpi = 1 units)
r_fm = {
    'variational'  : -1,
    'hex-dibaryon' : gv.gvar('1.04(21)'),                 # Eq. (19) in hex-dibaryon
    'physical'     : -1,
}[paper]                     # effective range (GeV)
r_mpi = {
    'variational'  : gv.gvar('4.34(2.24)'),
    'hex-dibaryon' : r_fm / hbarc * mpi,
    'physical'     : -1,
}[paper]                     # effective range (mpi = 1 units). Note the sdev might be 1/2 of this
mN_lat = {
    'variational'  : -1,
    'hex-dibaryon' : gv.gvar('1.2025(8)'),                # page 11 in hex-dibaryon
    'physical'     : -1,
}[paper]                     # nucleon mass (MeV). This is M in Zohreh's paper.
mN   = {
    'variational'  : gv.gvar('1.636(18)'),
    'hex-dibaryon' : mN_lat * (hbarc / a_lat),
    'physical'     : -1,
}[paper]                     # nucleon mass (MeV). This is M in Zohreh's paper.
delta_mNN = {
    'variational'  : gv.gvar('-3.33(68)') / 1000.0,
    'hex-dibaryon' : gv.gvar('20.6(3.3)') / 1000.0,        # Eq. (25) in hex-dibaryon
    'physical'     : -1,
}[paper]                     # dinucleon energy shift (GeV). Equals E in Zohreh's paper
# mNN  = {
#     'variational'  : -1,
#     'hex-dibaryon' : -1,
#     'physical'     : -1,
# }[paper]                     # dinucleon mass (GeV).
mNN = 2*mN + delta_mNN        # dinucleon mass (GeV).
mu = 0.770                    # physical rho mass (GeV) 

In [67]:
# class ComplexGVar:
    
#     def __init__(self, re, im):
#         self.data = np.array([gv.gvar(re), gv.gvar(im)], dtype = object)
#     def __add__(self, y):
#         return ComplexGVar(self.data[0] + y.data[0], self.data[1] + y.data[1])
# add not working...

In [10]:
# convert nucleon masses to Mpi = 1 units
mN_mpi = to_mpi_units(mN)
mNN_mpi = to_mpi_units(mNN)
delta_mNN_mpi = to_mpi_units(delta_mNN)

In [73]:
p = gv.sqrt(mN_mpi * gv.abs(delta_mNN))                  # Mpi = 1 units
pcotd = -ainv_mpi + (r_mpi * (p**2)) / 2         # Mpi = 1 units
M_LO_re = -(4*np.pi/mN_mpi) * ainv_mpi / (ainv_mpi**2 + p**2)
M_LO_im = (4*np.pi/mN_mpi) * p / (ainv_mpi**2 + p**2)

In [74]:
p

0.0822(84)

In [75]:
delta_mNN

-0.00333(68)

#### Shift $\delta J$

In [ ]:
# Value of deltaJ computed by Anthony, to be implemented and cross-checked later
# Note that one of hex-dibaryon or physical is incorrect.
dJ = {
    'variational'  : -366.5923642228772,
    'hex-dibaryon' : -4100.476281523836,
    'physical'     : -4100.476281523836,
}[paper]